In [4]:
import pandas as pd
import os
from utils.pandaUtils import standardize_col_names, convert_dt_cols, convert_numeric_cols, merge_all_df

# Global Variables

In [199]:
DATA_DIR = './data/raw'
OUTPUT_DIR = './data/output'
DT_FORMAT = "%d.%m.%Y"

# Purchase Data Generic Data Cleaning

In [200]:
pur_df = pd.read_csv(os.path.join(DATA_DIR, "purchases.csv"))
dt_cols = ['posting_date']
pur_df = standardize_col_names(pur_df)
pur_df = convert_dt_cols(pur_df, dt_cols, DT_FORMAT)
numeric_cols = ['quantity', 'price']
pur_df = convert_numeric_cols(pur_df, numeric_cols)

rename_dict = {"posting_date": "date", "canceled": "cancelled", "customervendor_code": "vendor_code", "customervendor_name": "vendor_name",
               "item_no": "item_code", "itemservice_description": "description", "uom_code": "uom_code", "price": "price", "quantity": "quantity"}

pur_df.rename(columns=rename_dict,inplace=True)
pur_df = pur_df[rename_dict.values()]

In [201]:
purchases = pd.read_csv(os.path.join(DATA_DIR, "purchases.csv"))
purchases['UoM Code'].unique()

array(['Manual', nan], dtype=object)

# DATA ERROR!

### Item codes sharing multiple descriptions in purchase data

In [202]:
req_cols = ['item_code','description']

mismatch_description = pur_df[req_cols].groupby('item_code').nunique().reset_index()
mismatch_description.rename(columns={"description":"unique_count"},inplace=True)

mismatch_pdt = list(mismatch_description[mismatch_description['unique_count'] > 1]['item_code'])
pur_df[pur_df['item_code'].isin(mismatch_pdt)][['item_code','description']].drop_duplicates().sort_values(by='item_code')

,item_code,description
2884,BFC001,BEEF FATS 25KG/CTNS (OAKEY-AUST)
1881,BFC001,BEEF FATS 25KG/CTNS
151,BKN003,BEEF KNUCKLE CUBE (TFP) 500gmX20pkt
2070,BKN003,BEEF KNUCKLE CUBE (SADIA) 500GM X20PKT
298,BMI005,BEEF- MINCED BEEF (TFP) 300gmX20pkt
...,...,...
1690,SVA005,SEAFOOD- VANNAMEI HOSO 31-40 12PKT/CTNS
895,SVA005,SEAFOOD- VANNAMEI HOSO 31-40 GOLDEN-PERU 700GM...
2582,SVA008-04,SEAFOOD- VANNAMEI HOSO 41-50pcs 10x1pkt/ctn
190,SVA008-04,SEAFOOD- VANNAMEI HOSO 41-50pcs A&O 10x1pkt/ctn


### No mismatched vendor names

In [203]:
req_cols = ['vendor_code','vendor_name']

mismatch_vendor_name = pur_df[req_cols].groupby('vendor_code').nunique().reset_index()
mismatch_vendor_name.rename(columns={"vendor_name":"unique_count"},inplace=True)
mismatch_vendors = list(mismatch_vendor_name[mismatch_vendor_name['unique_count'] > 1]['vendor_code'])
pur_df[pur_df['vendor_code'].isin(mismatch_vendors)][['vendor_code','vendor_name']].drop_duplicates().sort_values(by='vendor_code')

,vendor_code,vendor_name


## DATA ERROR!
### Descriptions sharing multiple item_codes in purchase data

In [204]:
pur_df['item_code'].fillna("Service", inplace=True)
pur_df['total'] = pur_df['quantity'] * pur_df['price']

pur_pdt_df = pur_df[pur_df['item_code'] != "Service"].copy()

# take the first description of each item_code
pur_pdt_df['description'] = pur_pdt_df.groupby(['item_code'])['description'].transform('first')

item_code_desc_cnt = pur_pdt_df[['description','item_code']].groupby('description')['item_code'].nunique().reset_index()

display(item_code_desc_cnt[item_code_desc_cnt['item_code'] > 1])
## 'FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 10X1KG/CTNS'
## 'OTHER- CHICKEN PATTIES BREADED (FARMPRIDE) 500gmX18pkt'

display(pur_pdt_df[pur_pdt_df['description'] == 'FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 10X1KG/CTNS'][['description','item_code']].drop_duplicates())
display(pur_pdt_df[pur_pdt_df['description'] == 'OTHER- CHICKEN PATTIES BREADED (FARMPRIDE) 500gmX18pkt'][['description','item_code']].drop_duplicates())

,description,item_code
199,FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 10...,2
272,OTHER- CHICKEN PATTIES BREADED (FARMPRIDE) 500...,2


,description,item_code
1858,FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 10...,FSN002-02
2361,FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 10...,FSN002-04


,description,item_code
724,OTHER- CHICKEN PATTIES BREADED (FARMPRIDE) 500...,OCP003
2378,OTHER- CHICKEN PATTIES BREADED (FARMPRIDE) 500...,OCP003-02


# Purchase Data Cleaning

In [205]:
pur_df = pur_df[pur_df['cancelled'] == 'N']
pur_df['item_code'].fillna("Service", inplace=True)
pur_df['total'] = pur_df['quantity'] * pur_df['price']
pur_pdt_df = pur_df[pur_df['item_code'] != "Service"].copy()

# take the first description of each item_code
pur_pdt_df['description'] = pur_pdt_df.groupby(['item_code'])['description'].transform('first')

# take the first item_code of each description
# pur_pdt_df['item_code'] = pur_pdt_df.groupby(['description'])['item_code'].transform('first')

# Product Purchase Exploration

### Product Summary

In [206]:
# how many suppliers does each item have?

req_cols = ['item_code', 'description', 'vendor_code']
num_suppliers = pur_pdt_df[req_cols].drop_duplicates().groupby(
    ['item_code', 'description'])['vendor_code'].nunique().reset_index()
num_suppliers.rename(columns={'vendor_code': 'num_suppliers'}, inplace=True)

req_cols = ['item_code', 'description', 'price', 'total', 'quantity', 'date']
base_df = pur_pdt_df[req_cols].groupby(['item_code', 'description'])[
    ['price']].describe().droplevel(level=0, axis=1).reset_index()
base_df.rename(columns={"count": "purchase_count", "mean": "avg_price",
               "min": "min_price", "50%": "median_price", "max": "max_price"}, inplace=True)
base_df.drop(columns=['std', '25%', '75%'], inplace=True)

total_pv = pur_pdt_df[req_cols].groupby(['item_code', 'description'])[
    'total'].sum().reset_index()
total_pv.rename(columns={"total": "total_pv"}, inplace=True)

total_qty = pur_pdt_df[req_cols].groupby(['item_code', 'description'])[
    'quantity'].sum().reset_index()
total_qty.rename(columns={"quantity": "total_qty"}, inplace=True)

latest_purchase_date_df = pur_pdt_df[req_cols].copy()
latest_purchase_date_df['latest_purchase_date'] = pur_pdt_df[req_cols].groupby(['item_code', 'description'])['date'].transform('max')

latest_purchase_price_df = latest_purchase_date_df[latest_purchase_date_df['date'] == latest_purchase_date_df['latest_purchase_date']].groupby(['item_code','description'])['price'].mean().reset_index()
latest_purchase_price_df.rename(columns={"price": "latest_purchase_price"}, inplace=True)

latest_purchase_date_df = latest_purchase_date_df[['item_code','description','latest_purchase_date']].drop_duplicates()

pdt_purchase_summary = merge_all_df(
    dfs=[latest_purchase_price_df, latest_purchase_date_df, total_qty, total_pv, num_suppliers, base_df], merge_keys=['item_code', 'description'])

In [207]:
pdt_purchase_summary

,item_code,description,purchase_count,avg_price,min_price,median_price,max_price,latest_purchase_price,latest_purchase_date,total_qty,total_pv,num_suppliers
0,BBA001,BEEF BALL 1kg/pkt (PING PONG),11.0,9.00,9.0,9.0,9.0,9.0,2022-12-31,700.00,6300.000,1
1,BBA001-01,BEEF & CHICKEN BALL - SWEDISH MEAT BALL 1KG/PKT,3.0,10.80,10.8,10.8,10.8,10.8,2022-12-01,30.00,324.000,1
2,BBR002,BEEF BRISKET PE * (QUALITY BEEF-BRAZIL),10.0,7.71,6.9,7.8,7.8,7.8,2022-12-31,1465.01,11402.193,2
3,BCHO03,BEEF CHOP (STRIPLOIN) STEAK (BUTHER'S RIDGE) 5...,3.0,7.60,7.6,7.6,7.6,7.6,2022-12-17,460.00,3496.000,1
4,BCHO05,BEEF MELTIQUE SIRLOIN STEAK 200GM/PCS,2.0,5.00,5.0,5.0,5.0,5.0,2022-11-19,546.00,2730.000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
590,SWC004,SEAFOOD- GONG GONG FRESH,1.0,5.00,5.0,5.0,5.0,5.0,2022-12-16,5.00,25.000,1
591,SWC004-01,SEAFOOD- GONG GONG 20kg,4.0,4.50,4.5,4.5,4.5,4.5,2022-12-29,580.00,2610.000,1
592,SWC005-01,SEAFOOD- COCKLES FRESH,2.0,5.00,5.0,5.0,5.0,5.0,2022-12-31,15.00,75.000,1
593,SWC006-02,SEAFOOD- BAMBOO CLAMS 10/12 10X1kg/ctn,7.0,4.00,4.0,4.0,4.0,4.0,2022-12-31,580.00,2320.000,1


### In Depth Product View (After Filtering)

In [208]:
# which suppliers are selling this product? (table view)
req_cols = ["item_code","description","vendor_name","price"]
pdt_suppliers_list = pur_pdt_df[req_cols].groupby(['item_code','description','vendor_name'])['price'].describe().reset_index()
pdt_suppliers_list.rename(columns={"count": "purchase_count", "mean": "avg_price",
               "min": "min_price", "50%": "median_price", "max": "max_price"}, inplace=True)
pdt_suppliers_list.drop(columns=['std', '25%', '75%'], inplace=True)
display(pdt_suppliers_list)

,item_code,description,vendor_name,purchase_count,avg_price,min_price,median_price,max_price
0,BBA001,BEEF BALL 1kg/pkt (PING PONG),PING PONG FOODS PTE LTD,11.0,9.0,9.0,9.0,9.0
1,BBA001-01,BEEF & CHICKEN BALL - SWEDISH MEAT BALL 1KG/PKT,PING PONG FOODS PTE LTD,3.0,10.8,10.8,10.8,10.8
2,BBR002,BEEF BRISKET PE * (QUALITY BEEF-BRAZIL),CASH SUPPLIER (NON-GST),1.0,6.9,6.9,6.9,6.9
3,BBR002,BEEF BRISKET PE * (QUALITY BEEF-BRAZIL),MEGA PACKERS ASSOCIATE PTE LTD,9.0,7.8,7.8,7.8,7.8
4,BCHO03,BEEF CHOP (STRIPLOIN) STEAK (BUTHER'S RIDGE) 5...,COUNTRY FOODS PTE LTD,3.0,7.6,7.6,7.6,7.6
...,...,...,...,...,...,...,...,...
761,SWC004,SEAFOOD- GONG GONG FRESH,TECK CASH SUPPLIER (NON-GST),1.0,5.0,5.0,5.0,5.0
762,SWC004-01,SEAFOOD- GONG GONG 20kg,LEE HUAT SEAFOOD SUPPLIER,4.0,4.5,4.5,4.5,4.5
763,SWC005-01,SEAFOOD- COCKLES FRESH,TECK CASH SUPPLIER (NON-GST),2.0,5.0,5.0,5.0,5.0
764,SWC006-02,SEAFOOD- BAMBOO CLAMS 10/12 10X1kg/ctn,LEE HUAT SEAFOOD SUPPLIER,7.0,4.0,4.0,4.0,4.0


In [209]:
# which suppliers are selling this product? (boxplot view)
req_cols = ["item_code","description","vendor_name","price"]
pdt_price_box_plt = pur_pdt_df[req_cols]

In [210]:
# time series of purchase price from all suppliers
req_cols = ["item_code", "description", "price", "date"]

time_series_price = pur_pdt_df[req_cols].copy()

# daily with upsampling
daily_price_upsample = time_series_price.groupby(['item_code', 'description']).resample(
    "D", on="date").median().reset_index()

# weekly with upsampling
weekly_price_upsample = time_series_price.groupby(['item_code', 'description']).resample(
    "W", on="date").median().reset_index()

# monthly with upsampling
monthly_price_upsample = time_series_price.groupby(['item_code', 'description']).resample(
    "MS", on="date").median().reset_index()

# adding date columns
time_series_price.loc[:, 'month_start'] = time_series_price['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
time_series_price.loc[:, 'week_start'] = time_series_price['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_price = time_series_price.groupby(
    ['item_code', 'description', 'date'])['price'].median().reset_index()

# weekly without upsampling
weekly_price = time_series_price.groupby(
    ['item_code', 'description', 'week_start'])['price'].median().reset_index()

# monthly without upsampling
monthly_price = time_series_price.groupby(
    ['item_code', 'description', 'month_start'])['price'].median().reset_index()

## Supplier Exploration

### Supplier Summary View

In [211]:
req_cols = ["vendor_code", "vendor_name", "total"]
base_df = pur_pdt_df[req_cols].groupby(['vendor_code', 'vendor_name'])[['total']].describe().droplevel(level=0,axis=1).reset_index()
base_df.rename(columns={"count": "purchase_count", "mean": "avg_pv", "min": "min_pv", "50%": "median_pv", "max": "max_pv"},inplace=True)
base_df.drop(columns=['std','25%','75%'], inplace=True)

# How much are we spending for each supplier?
total_pv = pur_pdt_df[req_cols].groupby(
    ["vendor_code", "vendor_name"]).sum().reset_index()
total_pv.rename(columns={'total': "total_pv"}, inplace=True)


req_cols = ['vendor_code', "vendor_name", "item_code"]
# what unique items are we buying from supplier?
unique_items_per_supplier_df = pur_pdt_df[req_cols].drop_duplicates()

# How many unique items are we buying from supplier?
unique_items_cnt = unique_items_per_supplier_df.groupby(
    ['vendor_code', 'vendor_name']).count().reset_index()
unique_items_cnt.rename(
    columns={'item_code': 'unique_items_count'}, inplace=True)
unique_items_cnt.sort_values(
    by="unique_items_count", inplace=True, ascending=False)

# how many items are ONLY sold by this supplier?
vendor_unique_item_list = pur_pdt_df.groupby(['vendor_code','vendor_name'])['item_code'].unique().reset_index()
vendor_unique_item_list.rename(columns={"item_code": "unique_items"},inplace=True)

no_of_suppliers_per_item = pur_pdt_df.groupby('item_code')['vendor_code'].nunique().reset_index()
no_of_suppliers_per_item.rename(columns={"vendor_code": "num_suppliers"},inplace=True)

items_with_one_supplier = list(no_of_suppliers_per_item[no_of_suppliers_per_item['num_suppliers'] == 1]['item_code'])

vendor_unique_item_list['num_only_supplier'] = vendor_unique_item_list['unique_items'].apply(lambda item_list: len(list(set(item_list) & set(items_with_one_supplier))))
only_supplier_count = vendor_unique_item_list[['vendor_code','vendor_name','num_only_supplier']]

supplier_summary = merge_all_df(
    dfs=[total_pv, unique_items_cnt, only_supplier_count, base_df], merge_keys=['vendor_code', 'vendor_name'])

display(supplier_summary)

,vendor_code,vendor_name,purchase_count,avg_pv,min_pv,median_pv,max_pv,total_pv,unique_items_count,num_only_supplier
0,USVS0018,SINGYI F&B PTE LTD,50.0,42.580000,12.30,35.55,192.00,2129.0000,4,3
1,UVA0001,ANGLISS SINGAPORE PTE LTD,56.0,1175.622784,9.00,676.80,5440.00,65834.8759,22,14
2,UVA0002,ALL BIG FROZEN FOOD PTE LTD,2.0,3190.000000,2900.00,3190.00,3480.00,6380.0000,1,1
3,UVA0003,ANG KEE FISH MERCHANT PTE LTD,1.0,130.000000,130.00,130.00,130.00,130.0000,1,1
4,UVA0004,ARCO MARKETING PTE LTD,7.0,653.714286,80.00,720.00,1440.00,4576.0000,2,1
...,...,...,...,...,...,...,...,...,...,...
119,UVT0042,TAO YUAN PTE LTD,20.0,138.962000,26.32,86.00,390.00,2779.2400,6,5
120,UVT0043,TW FOODSTUFF,1.0,53.000000,53.00,53.00,53.00,53.0000,1,1
121,UVX0002,XIN JIA FU FOOD PTE LTD,2.0,150.020000,150.02,150.02,150.02,300.0400,1,1
122,UVY0006,YOCORN FOOD ENTERPRISE PTE LTD,2.0,220.000000,50.00,220.00,390.00,440.0000,2,2


### In depth Supplier View (After Filtering)

In [212]:
# for time series of pv value
req_cols = ["vendor_code", "vendor_name",
            'date', 'item_code', 'description', 'total']

time_series_pv = pur_pdt_df[req_cols].copy()

# daily with upsampling
daily_pv_per_pdt_per_supplier_upsample = time_series_pv.groupby(['vendor_code', 'vendor_name', 'item_code', 'description']).resample(
    "D", on="date").sum().drop(columns=['vendor_code', 'vendor_name', 'item_code', 'description']).reset_index()

# weekly with upsampling
weekly_pv_per_pdt_per_supplier_upsample = time_series_pv.groupby(['vendor_code', 'vendor_name', 'item_code', 'description']).resample(
    "W", on="date").sum().drop(columns=['vendor_code', 'vendor_name', 'item_code', 'description']).reset_index()

# monthly with upsampling
monthly_pv_per_pdt_per_supplier_upsample = time_series_pv.groupby(['vendor_code', 'vendor_name', 'item_code','description']).resample(
    "MS", on="date").sum().drop(columns=['vendor_code', 'vendor_name', 'item_code','description']).reset_index()

# adding date columns
time_series_pv.loc[:, 'month_start'] = time_series_pv['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
time_series_pv.loc[:, 'week_start'] = time_series_pv['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_pv_per_pdt_per_supplier = time_series_pv.groupby(['item_code','description', 'date', 'vendor_code', 'vendor_name'])[
    'total'].sum().reset_index()

# weekly without upsampling
weekly_pv_per_pdt_per_supplier = time_series_pv.groupby(
    ['item_code','description', 'week_start', 'vendor_code', 'vendor_name'])['total'].sum().reset_index()

# monthly without upsampling
monthly_pv_per_pdt_per_supplier = time_series_pv.groupby(
    ['item_code','description', 'month_start', 'vendor_code', 'vendor_name'])['total'].sum().reset_index()

In [257]:
# for time series of pv value
req_cols = ["item_code", "description", "vendor_code", "vendor_name",
            'date', 'price']

ts_price_per_pdt_per_vendor = pur_pdt_df[req_cols].copy()

# daily with upsampling
daily_price_per_pdt_per_supplier_upsample = ts_price_per_pdt_per_vendor.groupby(["item_code", "description", "vendor_code", "vendor_name"]).resample(
    "D", on="date").sum().drop(columns=["item_code", "description", "vendor_code", "vendor_name"]).reset_index()

# weekly with upsampling
weekly_price_per_pdt_per_supplier_upsample = ts_price_per_pdt_per_vendor.groupby(["item_code", "description", "vendor_code", "vendor_name"]).resample(
    "W", on="date").sum().drop(columns=["item_code", "description", "vendor_code", "vendor_name"]).reset_index()

# monthly with upsampling
monthly_price_per_pdt_per_supplier_upsample = ts_price_per_pdt_per_vendor.groupby(["item_code", "description", "vendor_code", "vendor_name"]).resample(
    "MS", on="date").sum().drop(columns=["item_code", "description", "vendor_code", "vendor_name"]).reset_index()

# adding date columns
ts_price_per_pdt_per_vendor.loc[:, 'month_start'] = ts_price_per_pdt_per_vendor['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
ts_price_per_pdt_per_vendor.loc[:, 'week_start'] = ts_price_per_pdt_per_vendor['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_price_per_pdt_supplier = ts_price_per_pdt_per_vendor.groupby(['date', "item_code", "description", "vendor_code", "vendor_name"])[
    'price'].sum().reset_index()

# weekly without upsampling
weekly_price_per_pdt_supplier = ts_price_per_pdt_per_vendor.groupby(
    ['week_start', "item_code", "description", "vendor_code", "vendor_name"])['price'].sum().reset_index()

# monthly without upsampling
monthly_price_per_pdt_supplier = ts_price_per_pdt_per_vendor.groupby(
    ['month_start', "item_code", "description", "vendor_code", "vendor_name"])['price'].sum().reset_index()

,item_code,median_price,min_price,max_price
0,BBA001,9.0,9.0,9.0
1,BBA001-01,10.8,10.8,10.8
2,BBR002,7.8,6.9,7.8
3,BCHO03,7.6,7.6,7.6
4,BCHO05,5.0,5.0,5.0
...,...,...,...,...
590,SWC004,5.0,5.0,5.0
591,SWC004-01,4.5,4.5,4.5
592,SWC005-01,5.0,5.0,5.0
593,SWC006-02,4.0,4.0,4.0


In [ ]:
# for time series of pv value
req_cols = ["vendor_code", "vendor_name",
            'date', 'total']

time_series_pv = pur_pdt_df[req_cols].copy()

# daily with upsampling
daily_pv_per_supplier_upsample = time_series_pv.groupby(['vendor_code', 'vendor_name']).resample(
    "D", on="date").sum().drop(columns=['vendor_code', 'vendor_name']).reset_index()

# weekly with upsampling
weekly_pv_per_supplier_upsample = time_series_pv.groupby(['vendor_code', 'vendor_name']).resample(
    "W", on="date").sum().drop(columns=['vendor_code', 'vendor_name']).reset_index()

# monthly with upsampling
monthly_pv_per_supplier_upsample = time_series_pv.groupby(['vendor_code', 'vendor_name']).resample(
    "MS", on="date").sum().drop(columns=['vendor_code', 'vendor_name']).reset_index()

# adding date columns
time_series_pv.loc[:, 'month_start'] = time_series_pv['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
time_series_pv.loc[:, 'week_start'] = time_series_pv['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_pv_per_supplier = time_series_pv.groupby(['date', 'vendor_code', 'vendor_name'])[
    'total'].sum().reset_index()

# weekly without upsampling
weekly_pv_per_supplier = time_series_pv.groupby(
    ['week_start', 'vendor_code', 'vendor_name'])['total'].sum().reset_index()

# monthly without upsampling
monthly_pv_per_supplier = time_series_pv.groupby(
    ['month_start', 'vendor_code', 'vendor_name'])['total'].sum().reset_index()

In [277]:
# how much each supplier is supplying for each item
req_cols = ["item_code", "description", "vendor_code",
            "vendor_name", 'price', 'quantity', 'total']

base_df = pur_pdt_df[req_cols].groupby(['vendor_code', 'vendor_name', 'item_code', 'description'])[
    ['price']].describe().droplevel(level=0, axis=1).reset_index()
base_df.rename(columns={"count": "purchase_count", "mean": "avg_price",
               "min": "min_price", "50%": "median_price", "max": "max_price"}, inplace=True)
base_df.drop(columns=['std', '25%', '75%'], inplace=True)

# what is the total purchase value for this item from this supplier?
supplier_item_total_pv = pur_pdt_df[req_cols].groupby(
    ['vendor_code', 'vendor_name', 'item_code', 'description'])['total'].sum().reset_index()
supplier_item_total_pv.rename(columns={"price": "total_pv"}, inplace=True)

# what is the total quantity for this item from this supplier?
supplier_item_total_qty = pur_pdt_df[req_cols].groupby(
    ['vendor_code', 'vendor_name', 'item_code', 'description'])['quantity'].sum().reset_index()
supplier_item_total_qty.rename(columns={"quantity": "total_qty"}, inplace=True)

# when did we last purchase these product from this supplier?
req_cols = ["item_code", "description", "vendor_code",
            "vendor_name", 'date']

supplier_item_latest_purchase_date = pur_pdt_df[req_cols].groupby(
    ['vendor_code', 'vendor_name', 'item_code', 'description'])['date'].max().reset_index()
supplier_item_latest_purchase_date.rename(
    columns={"date": "latest_purchase_date"}, inplace=True)

supplier_items_summary = merge_all_df(
    dfs=[supplier_item_total_pv, supplier_item_total_qty, supplier_item_latest_purchase_date, base_df], merge_keys=['vendor_code', 'vendor_name', 'item_code', 'description'])

### Comparing individual supplier prices to group item prices

In [278]:
req_cols = ['item_code', 'median_price', 'max_price', 'latest_purchase_price','num_suppliers']

group_pdt = pdt_purchase_summary[req_cols].rename(
    columns={"median_price": "group_median_price",  "max_price": "group_max_price", "latest_purchase_price": "group_latest_price"})

supplier_items_summary = supplier_items_summary.merge(group_pdt, on='item_code', how='left')
supplier_items_summary['price_diff_from_group'] = supplier_items_summary['median_price'] - supplier_items_summary['group_median_price']
display(supplier_items_summary)

,vendor_code,vendor_name,item_code,description,purchase_count,avg_price,min_price,median_price,max_price,total,total_qty,latest_purchase_date,group_median_price,group_max_price,group_latest_price,num_suppliers,price_diff_from_group
0,USVS0018,SINGYI F&B PTE LTD,MECHICKEN03,MARINATE CHICKEN MID JOINT 5KG/PKT,10.0,7.800000,7.80,7.80,7.80,390.0,50.0,2022-12-26,7.80,7.80,7.80,1,0.00
1,USVS0018,SINGYI F&B PTE LTD,MEPORK01,MARINATE PORK BELLY,4.0,5.700000,5.70,5.70,5.70,159.6,28.0,2022-12-06,5.70,5.70,5.70,1,0.00
2,USVS0018,SINGYI F&B PTE LTD,OPN001,OTHER- PRAWN NUTS 1kg/pkt,31.0,12.509677,12.30,12.30,12.80,1484.4,118.0,2022-12-31,12.30,12.80,12.80,1,0.00
3,USVS0018,SINGYI F&B PTE LTD,OYR001,OTHER- FRIED YAM RING 230gm,5.0,3.800000,3.80,3.80,3.80,95.0,25.0,2022-12-29,3.80,4.00,4.00,3,0.00
4,UVA0001,ANGLISS SINGAPORE PTE LTD,BFC005,BEEF FATS 20KG/CTNS (HARVEY-AUST),1.0,3.600000,3.60,3.60,3.60,360.0,100.0,2022-10-29,3.55,3.60,3.50,2,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,UVY0006,YOCORN FOOD ENTERPRISE PTE LTD,OWS0002,OTHER- WASABI MINI PKT 2.6GM/SACHETS,1.0,0.050000,0.05,0.05,0.05,50.0,1000.0,2022-11-23,0.05,0.05,0.05,1,0.00
762,UVY0006,YOCORN FOOD ENTERPRISE PTE LTD,SCA001-07,SEAFOOD- SCALLOP HOTATE KAIBASHIDA SIZE 3S,1.0,39.000000,39.00,39.00,39.00,390.0,10.0,2022-10-12,39.00,39.00,39.00,1,0.00
763,UVZ0003,ZUZEN FOOD PTE LTD,OCL014,OTHER- MOUSSE CAKE - RED VELVET,1.0,20.000000,20.00,20.00,20.00,20.0,1.0,2022-12-24,20.00,20.00,20.00,1,0.00
764,UVZ0003,ZUZEN FOOD PTE LTD,SCP001,SEAFOOD- CUTTLEFISH PASTE 1kg x 30pkt,9.0,5.900000,5.70,5.90,6.50,5546.0,940.0,2022-12-24,5.90,6.50,5.90,1,0.00


# Sales Data Generic Cleaning

In [216]:
sales_df = pd.read_csv(os.path.join(DATA_DIR, "sales.csv"))
dt_cols = ['posting_date']
sales_df = standardize_col_names(sales_df)
sales_df = convert_dt_cols(sales_df, dt_cols, DT_FORMAT)
numeric_cols = ['quantity']
sales_df = convert_numeric_cols(sales_df, numeric_cols)

rename_dict = {"posting_date": "date",
               "item_description": "description"}
req_cols = ['date', 'customer_code', 'customer_name', 'item_code',
            'description', 'quantity', 'currency', 'selling_price','document_no']

sales_df.rename(columns=rename_dict,inplace=True)
sales_df = sales_df[req_cols]

# Errors in Sales Data

### Item code got multiple descriptions (same as purchasing data)

In [217]:
req_cols = ['item_code','description']

mismatch_description = sales_df[req_cols].groupby('item_code').nunique().reset_index()
mismatch_description.rename(columns={"description":"unique_count"},inplace=True)

mismatch_pdt = list(mismatch_description[mismatch_description['unique_count'] > 1]['item_code'])
sales_df[sales_df['item_code'].isin(mismatch_pdt)][['item_code','description']].drop_duplicates().sort_values(by='item_code')

,item_code,description
56888,BFC001,BEEF FATS 25KG/CTNS
62223,BFC001,BEEF FATS 25KG/CTNS (OAKEY-AUST)
61331,BKN003,BEEF KNUCKLE CUBE (SADIA) 500GM X20PKT
1267,BKN003,BEEF KNUCKLE CUBE (TFP) 500gmX20pkt
59825,BMI005,BEEF- MINCED BEEF (SADIA) 300GM X 20PKT
...,...,...
31,SVA005,SEAFOOD- VANNAMEI HOSO 31-40 GOLDEN-PERU 700GM...
38356,SVA005,SEAFOOD- VANNAMEI HOSO 31-40 12PKT/CTNS
72556,SVA008-04,SEAFOOD- VANNAMEI HOSO 41-50pcs 12x1pkt/ctn
897,SVA008-04,SEAFOOD- VANNAMEI HOSO 41-50pcs A&O 10x1pkt/ctn


### Customer Code got multiple customer names

In [218]:
req_cols = ['customer_code','customer_name']

mismatch_customer_name = sales_df[req_cols].groupby('customer_code').nunique().reset_index()
mismatch_customer_name.rename(columns={"customer_name":"unique_count"},inplace=True)
mismatch_customers = list(mismatch_customer_name[mismatch_customer_name['unique_count'] > 1]['customer_code'])
sales_df[sales_df['customer_code'].isin(mismatch_customers)][['customer_code','customer_name']].drop_duplicates().sort_values(by='customer_code')

,customer_code,customer_name
6697,UCA0154,ACE MART PTE LTD (465NB)
128909,UCA0154,ACE MARKETPLACE PTE. LTD
98884,UCC0051,THYE GUAN EATING HOUSE
567,UCC0051,CHICKEN HOUSE PTE LTD
3057,UCC0203,CHIA YUK HIANG COFFEE HOUSE PTE LTD
46100,UCC0203,BUGIS XIN YUAN JI PTE LTD
44265,UCC0205,CHICKEN RICE (TAMPINES 3)
16079,UCC0205,CHICKEN RICE (TAMPINES 1)
13440,UCD0051,DA LONG YI FOOD & BEVERAGE MANAGEMENT PTE LTD
88555,UCD0051,SINGAPORE DA LONG YI FOOD & BEVERAGE MANAGEMEN...


## DATA ERROR!
### Descriptions sharing two item_codes in sales_data (same as purchase data)

In [219]:
item_code_desc_cnt = sales_df[['description','item_code']].groupby('description')['item_code'].nunique().reset_index()

display(item_code_desc_cnt[item_code_desc_cnt['item_code'] > 1])

display(sales_df[sales_df['description'] == "BEEF KNUCKLE CUBE (BUTHER'S RIDGE) 500GMX20PKT"][['description','item_code']].drop_duplicates())
display(sales_df[sales_df['description'] == 'PORK COLLAR CS IBERICO STEAK CUT 20PK/CTN'][['description','item_code']].drop_duplicates())

,description,item_code
28,BEEF KNUCKLE CUBE (BUTHER'S RIDGE) 500GMX20PKT,2
208,ChICKEN WING STICK(SEARA) 12x1KG/CTNS,2
314,FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 10...,2
346,MUTTON- LAMB FLAP BONE (WOODWARD),2
410,OTHER- CHICKEN PATTIES (PRIMO-ROSEMARY) 420gmX...,2
411,OTHER- CHICKEN PATTIES BREADED (FARMPRIDE) 500...,2
641,PORK COLLAR CS IBERICO STEAK CUT 20PK/CTN,2
737,PORK RECTUMS 10kg/ctns (DAT SCHAUB DENMARK),2


,description,item_code
55320,BEEF KNUCKLE CUBE (BUTHER'S RIDGE) 500GMX20PKT,BKN005
78254,BEEF KNUCKLE CUBE (BUTHER'S RIDGE) 500GMX20PKT,BKN004


,description,item_code
18884,PORK COLLAR CS IBERICO STEAK CUT 20PK/CTN,PCOL10-03
127812,PORK COLLAR CS IBERICO STEAK CUT 20PK/CTN,PCOL10-02


### One customer name sharing multiple customer codes

In [220]:
customer_code_name_cnt = sales_df[['customer_code','customer_name']].groupby('customer_name')['customer_code'].nunique().reset_index()

display(customer_code_name_cnt[customer_code_name_cnt['customer_code'] > 1])

display(sales_df[sales_df['customer_name'] == "365 FOOD SERVICES PTE LTD"][['customer_name','customer_code']].drop_duplicates())
display(sales_df[sales_df['customer_name'] == "ZHU JIA"][['customer_name','customer_code']].drop_duplicates())

,customer_name,customer_code
6,365 FOOD SERVICE PTE LTD,3
7,365 FOOD SERVICES PTE LTD,4
17,6IX 6IX RESTAURANT AND BAR PTE LTD,3
18,7 DAYS COFFEE PTE LTD,2
21,8 BAR PTE LTD,4
...,...,...
1022,YUNXIN PTE LTD,3
1023,YY MEISHI PTE LTD,2
1030,ZHEN SI CHUAN MALA HOTPOT,2
1032,ZHONG GUO FENG PTE LTD,2


,customer_name,customer_code
564,365 FOOD SERVICES PTE LTD,UCN0015
703,365 FOOD SERVICES PTE LTD,UCN0053
1998,365 FOOD SERVICES PTE LTD,UCD0007
107561,365 FOOD SERVICES PTE LTD,UCD0152


,customer_name,customer_code
1334,ZHU JIA,UCM0021
1613,ZHU JIA,UCZ0118
2790,ZHU JIA,UCZ0052
3948,ZHU JIA,UCZ0051
8054,ZHU JIA,UCZ0106
30053,ZHU JIA,UCZ0068
67340,ZHU JIA,UCZ0129
118225,ZHU JIA,UCZ0107
127428,ZHU JIA,UCZ0130


# Sales Data Cleaning

In [221]:
# take the first description of each item_code
sales_df['description'] = sales_df.groupby(['item_code'])['description'].transform('first')

# # take the first item_code of each description
# sales_df['item_code'] = sales_df.groupby(['description'])['item_code'].transform('first')

# take the first customer name of each customer_code
sales_df['customer_name'] = sales_df.groupby(['customer_code'])['customer_name'].transform('first')

# # take the first customer code of each customer name
# sales_df['customer_code'] = sales_df.groupby(['customer_name'])['customer_code'].transform('first')

# filtering out selling price = 0.01 for samples & all item_codes with zs that are not products
cond = ((sales_df['selling_price'] > 0.01) & ~sales_df['item_code'].str.contains('ZS'))
sales_df = sales_df[cond]

sales_df['total'] = sales_df['quantity'] * sales_df['selling_price']

sales_with_cogs = sales_df.merge(pdt_purchase_summary[['item_code', 'latest_purchase_price',
               'median_price', 'avg_price','min_price','max_price']], on='item_code', how='inner')

In [222]:
sales_with_cogs['latest_profit_per_uom'] = sales_with_cogs['selling_price'] - sales_with_cogs['latest_purchase_price']
sales_with_cogs['median_profit_per_uom'] = sales_with_cogs['selling_price'] - sales_with_cogs['median_price']
sales_with_cogs['avg_profit_per_uom'] = sales_with_cogs['selling_price'] - sales_with_cogs['avg_price']
sales_with_cogs['max_profit_per_uom'] = sales_with_cogs['selling_price'] - sales_with_cogs['min_price']
sales_with_cogs['min_profit_per_uom'] = sales_with_cogs['selling_price'] - sales_with_cogs['max_price']

sales_with_cogs['latest_total_profit'] = sales_with_cogs['latest_profit_per_uom'] * sales_with_cogs['quantity']
sales_with_cogs['median_total_profit'] = sales_with_cogs['median_profit_per_uom'] * sales_with_cogs['quantity']
sales_with_cogs['avg_total_profit'] = sales_with_cogs['avg_profit_per_uom'] * sales_with_cogs['quantity']
sales_with_cogs['max_total_profit'] = sales_with_cogs['max_profit_per_uom'] * sales_with_cogs['quantity']
sales_with_cogs['min_total_profit'] = sales_with_cogs['min_profit_per_uom'] * sales_with_cogs['quantity']

sales_with_cogs['latest_pc1'] =  sales_with_cogs['latest_profit_per_uom'] / sales_with_cogs['selling_price']
sales_with_cogs['median_pc1'] =  sales_with_cogs['median_profit_per_uom'] / sales_with_cogs['selling_price']
sales_with_cogs['avg_pc1'] =  sales_with_cogs['avg_profit_per_uom'] / sales_with_cogs['selling_price']
sales_with_cogs['max_pc1'] =  sales_with_cogs['max_profit_per_uom'] / sales_with_cogs['selling_price']
sales_with_cogs['min_pc1'] =  sales_with_cogs['min_profit_per_uom'] / sales_with_cogs['selling_price']

## ERROR!

### Missing Item Codes In Purchase Data that exist in Sales Data

Possible Reasons:
- Could be just products were purchased way before
- Data error in item codes

In [197]:
check_sales = sales_df.merge(pdt_purchase_summary[['item_code', 'latest_purchase_price',
               'median_price', 'avg_price']], on='item_code', how='left')

missing_sales_item_codes = check_sales[check_sales['latest_purchase_price'].isna()][['item_code','description']].drop_duplicates()
display(missing_sales_item_codes)

,item_code,description
0,PS-02,PORK LEG SLICED 2KGX6PKT/CTNS
6,BGC001,BEEF- BEEF GOLDEN COIN MUSCLE (MINERVA-BRAZIL)
8,PRIB01-13,PORK MEALTY RIBLETS (AURORA) 10kg/ctn
9,PM-01,PORK- MINCED PORK 6X2KG/CTNS
18,BMS003,BEEF MEAT SLICED 2-3MM 2kgX5pkt
...,...,...
114848,SBT002,SEAFOOD- BLACK TIGER PTO 26/30 1kgX10pkt
116055,FSF006-03,FISH- SUTCHI STEAK 80-100gm (TWIN FISH/BIG FIS...
119449,OOI004-01,OTHER- COOKING CHINESE WINE FINE COOK
123073,PMPD01,PORK MEAT POWDER


## ERROR!

### Missing Item Codes In Sales Data that exist in Purchase Data
Possible Reasons:
- Could be just they were not sold at all during this time period
- Data error in item codes

In [198]:
check_purchase = pdt_purchase_summary[['item_code', 'latest_purchase_price',
               'median_price', 'avg_price']].merge(sales_df, on='item_code', how='left')

missing_purchase_item_codes = check_purchase[check_purchase['date'].isna()][['item_code','description']].drop_duplicates()
display(missing_purchase_item_codes)

,item_code,description
1486,BLO001,NaN
28581,MTU005,NaN
28770,OAB004,NaN
30669,OBA011,NaN
30672,OBOX001,NaN
41403,OPR003-02,NaN
48198,PBAK05,NaN
49909,PBEL05-03,NaN
50800,PBEL05-17,NaN
50801,PBEL05-20,NaN


# Sales Data Exploration

### Product Summary View

In [225]:

req_cols = ["item_code", 'description', 'selling_price']

base_df = sales_df[req_cols].groupby(['item_code', 'description']).describe(
).droplevel(level=0, axis=1).reset_index()
base_df.rename(columns={"mean": "avg_sales_price", "min": "min_sales_price",
               "50%": "median_sales_price", "max": "max_sales_price"}, inplace=True)
base_df.drop(columns=["count", 'std', '25%', '75%'], inplace=True)

total_revenue = sales_df.groupby(["item_code", 'description'])[
    'total'].sum().reset_index()
total_revenue.rename(columns={"total": "total_revenue"}, inplace=True)

num_invoices = sales_df.groupby(["item_code", 'description'])[
    'document_no'].nunique().reset_index()
num_invoices.rename(columns={"document_no": "num_invoices"}, inplace=True)

total_qty = sales_df.groupby(["item_code", 'description'])[
    'quantity'].sum().reset_index()
total_qty.rename(columns={"quantity": "total_sales_qty"}, inplace=True)

num_unique_customers = sales_df.groupby(["item_code", 'description'])[
    'customer_code'].nunique().reset_index()
num_unique_customers.rename(
    columns={"customer_code": "num_unique_customers"}, inplace=True)

total_profit = sales_with_cogs.groupby(["item_code", 'description'])[
    'median_total_profit'].sum().reset_index()
total_profit.rename(
    columns={"median_total_profit": "total_profit"}, inplace=True)

neg_pc1 = sales_with_cogs[sales_with_cogs['median_pc1'] < 0].groupby(
    ["item_code", "description"])['median_total_profit'].sum().reset_index()
neg_pc1.rename(columns={"median_total_profit": "pc1_loss"}, inplace=True)
neg_pc1["pc1_loss"] = abs(neg_pc1["pc1_loss"])

neg_pc1_cnt = sales_with_cogs[sales_with_cogs['median_pc1'] < 0].groupby(
    ["item_code", "description"])['document_no'].nunique().reset_index()
neg_pc1_cnt.rename(
    columns={"document_no": "pc1_loss_invoice_cnt"}, inplace=True)

req_cols = ['item_code', 'description', 'date', 'selling_price']

latest_sales_date_df = sales_df[req_cols].copy()
latest_sales_date_df['latest_sales_date'] = sales_df[req_cols].groupby(
    ['item_code', 'description'])['date'].transform('max')

latest_sales_price_df = latest_sales_date_df[latest_sales_date_df['date'] == latest_sales_date_df['latest_sales_date']].groupby(
    ['item_code', 'description'])['selling_price'].median().reset_index()
latest_sales_price_df.rename(
    columns={"selling_price": "latest_sales_price"}, inplace=True)

req_cols = ["item_code", 'description', 'median_pc1']
pc1 = sales_with_cogs[req_cols].groupby(['item_code', 'description']).describe(
).droplevel(level=0, axis=1).reset_index()
pc1.rename(columns={"mean": "avg_pc1", "min": "min_pc1",
                    "50%": "median_pc1", "max": "max_pc1"}, inplace=True)
pc1.drop(columns=["count", 'std', '25%', '75%'], inplace=True)

latest_sales_date_df = latest_sales_date_df[['item_code', 'description','latest_sales_date']].drop_duplicates()

pdt_sales_summary = merge_all_df([latest_sales_price_df, latest_sales_date_df, total_revenue, total_qty, total_profit, pc1, neg_pc1, neg_pc1_cnt,
                                  num_invoices, num_unique_customers, base_df], merge_keys=['item_code', 'description'])

### In depth Product Sales View (After Filtering)

In [226]:
# time series sales quantity

req_cols = ['date', 'item_code', 'description', 'quantity']
time_series_sales_qty = sales_df[req_cols].copy()

# daily with upsampling
daily_sales_qty_upsample = time_series_sales_qty.groupby(['item_code', 'description']).resample(
    "D", on="date").sum().drop(columns=['item_code', 'description']).reset_index()

# weekly with upsampling
weekly_sales_qty_upsample = time_series_sales_qty.groupby(['item_code', 'description']).resample(
    "W", on="date").sum().drop(columns=['item_code', 'description']).reset_index()

# monthly with upsampling
monthly_sales_qty_upsample = time_series_sales_qty.groupby(['item_code', 'description']).resample(
    "MS", on="date").sum().drop(columns=['item_code', 'description']).reset_index()

# adding date columns
time_series_sales_qty.loc[:, 'month_start'] = time_series_sales_qty['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
time_series_sales_qty.loc[:, 'week_start'] = time_series_sales_qty['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_sales_qty = time_series_sales_qty.groupby(
    ['item_code', 'description', 'date'])['quantity'].sum().reset_index()

# weekly without upsampling
weekly_sales_qty = time_series_sales_qty.groupby(
    ['item_code', 'description', 'week_start'])['quantity'].sum().reset_index()

# monthly without upsampling
monthly_sales_qty = time_series_sales_qty.groupby(
    ['item_code', 'description', 'month_start'])['quantity'].sum().reset_index()

In [227]:
profit_loss_invoices = sales_with_cogs[sales_with_cogs['median_pc1'] < 0]

In [228]:
avg_monthly_sales_qty = monthly_sales_qty_upsample.groupby(['item_code','description'])['quantity'].mean().reset_index()
avg_monthly_sales_qty.rename(columns={"quantity": "avg_monthly_sales_qty"},inplace=True)
pdt_sales_summary = pdt_sales_summary.merge(avg_monthly_sales_qty, how='left', on=["item_code","description"])


In [73]:
# time series sales price

req_cols = ['date', 'item_code', 'description', 'selling_price']
time_series_sales_price = sales_df[req_cols].copy()

# daily with upsampling
daily_sales_price_upsample = time_series_sales_price.groupby(['item_code', 'description']).resample(
    "D", on="date").median().reset_index()

# weekly with upsampling
weekly_sales_price_upsample = time_series_sales_price.groupby(['item_code', 'description']).resample(
    "W", on="date").median().reset_index()

# monthly with upsampling
monthly_sales_price_upsample = time_series_sales_price.groupby(['item_code', 'description']).resample(
    "MS", on="date").median().reset_index()

# adding date columns
time_series_sales_price.loc[:, 'month_start'] = time_series_sales_price['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
time_series_sales_price.loc[:, 'week_start'] = time_series_sales_price['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_sales_price = time_series_sales_price.groupby(
    ['item_code', 'description', 'date'])['selling_price'].median().reset_index()

# weekly without upsampling
weekly_sales_price = time_series_sales_price.groupby(
    ['item_code', 'description', 'week_start'])['selling_price'].median().reset_index()

# monthly without upsampling
monthly_sales_price = time_series_sales_price.groupby(
    ['item_code', 'description', 'month_start'])['selling_price'].median().reset_index()

In [74]:
req_cols = ['date', 'item_code', 'description', 'selling_price',
            'quantity', 'customer_code', 'customer_name']

price_df = sales_df[req_cols].groupby(['item_code', 'description', 'customer_code', 'customer_name'])[['selling_price']].describe(
).droplevel(level=0, axis=1).reset_index()
price_df.rename(columns={"mean": "avg_sales_price", "min": "min_sales_price",
                         "50%": "median_sales_price", "max": "max_sales_price"}, inplace=True)
price_df.drop(columns=["count", 'std', '25%', '75%'], inplace=True)

qty_df = sales_df[req_cols].groupby(['item_code', 'description', 'customer_code', 'customer_name'])[
    ['quantity']].sum().reset_index()
qty_df.rename(columns={"quantity": "total_qty"}, inplace=True)

latest_sales_date_df = sales_df[req_cols].groupby(
    ['item_code', 'description', 'customer_code', 'customer_name'])['date'].max().reset_index()
latest_sales_date_df.rename(
    columns={"date": "latest_sales_date"}, inplace=True)

latest_sales_price_df = sales_df.merge(latest_sales_date_df, how='left', on=[
                                       'item_code', 'description', 'customer_code', 'customer_name']).copy()
latest_sales_price_df = latest_sales_price_df[latest_sales_price_df['date']
                                              == latest_sales_price_df['latest_sales_date']]
latest_sales_price_df = latest_sales_price_df.groupby(
    ['item_code', 'description', 'customer_code', 'customer_name'])['selling_price'].median().reset_index()
latest_sales_price_df.rename(
    columns={"selling_price": "latest_sales_price"}, inplace=True)

pdt_sales_summary_per_customer = merge_all_df(dfs=[qty_df, latest_sales_date_df, latest_sales_price_df, price_df], merge_keys=[
    'item_code', 'description', 'customer_code', 'customer_name'])

In [75]:
req_cols = ['item_code', 'description','total', 'customer_code', 'customer_name','date']
time_series_sales_total_per_customer = sales_df[req_cols].copy()

# daily with upsampling
daily_sales_total_per_customer_upsample = time_series_sales_total_per_customer.groupby(['item_code', 'description','customer_code','customer_name']).resample(
    "D", on="date").median().reset_index()

# weekly with upsampling
weekly_sales_total_per_customer_upsample = time_series_sales_total_per_customer.groupby(['item_code', 'description','customer_code','customer_name']).resample(
    "W", on="date").median().reset_index()

# monthly with upsampling
monthly_sales_total_per_customer_upsample = time_series_sales_total_per_customer.groupby(['item_code', 'description','customer_code','customer_name']).resample(
    "MS", on="date").median().reset_index()

# adding date columns
time_series_sales_total_per_customer.loc[:, 'month_start'] = time_series_sales_total_per_customer['date'].apply(
    lambda ts: pd.offsets.MonthBegin().rollback(ts))
time_series_sales_total_per_customer.loc[:, 'week_start'] = time_series_sales_total_per_customer['date'] + \
    pd.offsets.Week(n=-1, weekday=0)

# daily without upsampling
daily_sales_total_per_customer = time_series_sales_total_per_customer.groupby(
    ['item_code', 'description','customer_code','customer_name', 'date'])['total'].median().reset_index()

# weekly without upsampling
weekly_sales_total_per_customer = time_series_sales_total_per_customer.groupby(
    ['item_code', 'description','customer_code','customer_name', 'week_start'])['total'].median().reset_index()

# monthly without upsampling
monthly_sales_total_per_customer = time_series_sales_total_per_customer.groupby(
    ['item_code', 'description','customer_code','customer_name', 'month_start'])['total'].median().reset_index()


# Inventory Data Generic Data Cleaning

In [240]:
inv_df = pd.read_csv(os.path.join(DATA_DIR, "cur_inventory.csv"))
inv_df = standardize_col_names(inv_df)
dt_cols = ['last_purchase_date']
inv_df = convert_dt_cols(inv_df, dt_cols, DT_FORMAT)
numeric_cols = ['actual_stock', 'total_value']
inv_df = convert_numeric_cols(inv_df, numeric_cols)

rename_dict = {"item_name": "description",
               "actual_stock" : "inv_qty",
               'item_no': 'item_code', 'total_value': 'total_inv_value'}

inv_df.rename(columns=rename_dict, inplace=True)
req_cols = ['item_code', 'description', 'inv_qty']
inv_df = inv_df[req_cols]

### Item Code Only Has 1 descripton! :)

In [241]:
req_cols = ['item_code','description']

mismatch_description = inv_df[req_cols].groupby('item_code').nunique().reset_index()
mismatch_description.rename(columns={"description":"unique_count"},inplace=True)

mismatch_pdt = list(mismatch_description[mismatch_description['unique_count'] > 1]['item_code'])
inv_df[inv_df['item_code'].isin(mismatch_pdt)][['item_code','description']].drop_duplicates().sort_values(by='item_code')

,item_code,description


# ERROR!

### Last purchase date is in 2023? Isn't this supposed to be inventory data as of 2023-12-31?

In [244]:
# display(inv_df[inv_df['last_purchase_date'] > "2022-12-31"])

# ERROR!

### Item description has multiple item codes (similar to purchasing and sales data)

In [243]:
multiple_item_codes = inv_df[req_cols].groupby('description').nunique().reset_index()
display(multiple_item_codes[multiple_item_codes['item_code'] > 1])

display(inv_df[inv_df['description'] == 'CHICKEN WING 90gm UP (NAT) 2kgX6pkt'][['description','item_code']].drop_duplicates())
display(inv_df[inv_df['description'] == 'PORK COLLAR SLICED 1CM'][['description','item_code']].drop_duplicates())

,description,item_code
118,CHICKEN WING 90gm UP (NAT) 2kgX6pkt,2
126,CHICKEN- MID JOINT 30gm (PERDIGAO) 1kgX12pkt,2
143,ChICKEN WING STICK(SEARA) 12x1KG/CTNS,2
219,FISH- TOMAN FILLET CUT SLICED (SKIN-ON) 4MM 1k...,2
294,OTHER- CHICKEN FRIED CRISPY RED HOT (AYAMAS) 8...,2
528,PORK COLLAR SLICED 1CM,2
569,PORK LEG CUT EXTRA SMALL CUBE 2kg/pkt,2


,description,item_code
179,CHICKEN WING 90gm UP (NAT) 2kgX6pkt,CWIN02-08
292,CHICKEN WING 90gm UP (NAT) 2kgX6pkt,CWIN02-06


,description,item_code
436,PORK COLLAR SLICED 1CM,PCOS04-01
642,PORK COLLAR SLICED 1CM,PCOS04-07


### Each item_code only has 1 row which is correct! :)

In [233]:
check_one_row = inv_df.groupby('item_code')['last_purchase_date'].count().reset_index()
display(check_one_row[check_one_row['last_purchase_date'] > 1])

,item_code,last_purchase_date


# ERROR! 
### Negative values in inventory

### Possibilities:
1. Products sold and substracted from inventory before delivery
2. Products purchased but not yet in warehouse

In [234]:
display(inv_df[inv_df['inv_qty'] < 0])

,item_code,description,inv_qty,last_purchase_date
40,BFQ002,BEEF FOREQUARTER B/LESS (SWIFT-HALAL),-70.0,2023-05-10
95,PBRB04-02,PORK BRISKET BONE (SMITHFIELD-USA) 13kg/ctn,-2.0,2023-05-10
109,BKN001-01,BEEF KNUCKLE (SWIFT-SA),-133.0,2023-06-20
124,OPN001,OTHER- PRAWN NUTS 1kg/pkt,-1.0,2023-06-04
125,OPO011,OTHER- SUPER CRISPY CHICKEN (CS TAY) 6X1kg,-12.0,2023-06-07
177,BUNDLE-02,SET B $48 - ABALONE 8's 1cans + BAK KU...,-34.0,NaT
193,PBAK01,TRADITIONAL BBQ BAK GUA 500gm/pkt,-3.0,2023-06-15
208,BUNDLE-03,SET B $48 - ABALONE 10's 1cans + BAK K...,-3.0,NaT
233,PLOI01,PORK LOIN PIF PAF/MABELLA 18kg/ctn,-658.0,2023-05-16
237,PTRI02,PORK TRIMMING FRESH,-232.9,2023-06-21


# Item codes in inventory data missing in sales data
### Possibilities:
1. Product did not sell in the past 3 months and there is still inventory from 3 months before
2. Product was purchased recently thus having inventory but no sales

In [235]:
check_inv_item_codes_missing_in_sales = inv_df.merge(sales_df,on='item_code',how='left')
display(check_inv_item_codes_missing_in_sales[check_inv_item_codes_missing_in_sales['date'].isna()][['item_code','description_x','last_purchase_date']].drop_duplicates())

,item_code,description_x,last_purchase_date
1389,FSB002-02,FISH- FRESH SEABASS 700-800GM (MALAYSIA),2022-04-07
1738,MLS004,MUTTON- LAMB RACK STANDARD (THOMAS-AUST),2021-08-29
3273,PEAR03,PORK EARFLAPS (WITH HAIR - ONLY EARFLAP),2022-03-14
4228,PLOI08-02,PORK LOIN 8mm 5kg/pkt,2022-05-31
4716,SAUCE-20B,SWISS BEAR TOMATO SACHET 1500PCS X 9GM,2021-08-10
...,...,...,...
121533,OOI011-02,OTHER- BLACK PEPPER POWDER 500GM/PKT,2021-07-19
121536,OOI017,OTHER- HONEY GARLIC SEASONING 12X1KG/CTNS,2020-06-16
121634,OSC004,OTHER- SEAWEED CHICKEN 1kgX10pkt,2022-06-10
121641,OTS004,OTHER - CHICKEN LUNCHEON MEAT KIZMIQ 340GM X 2...,2022-05-04


# Item codes in inventory data missing in purchase data
### Possibilities:
1. Did not purchase any of these products recently but still have inventory meaning these products are in the warehouse for at least 3 months

In [236]:
check_inv_item_codes_missing_in_purchases = inv_df.merge(pur_pdt_df,on='item_code',how='left')
display(check_inv_item_codes_missing_in_purchases[check_inv_item_codes_missing_in_purchases['date'].isna()][['item_code','description_x','last_purchase_date']].drop_duplicates())


,item_code,description_x,last_purchase_date
10,BCT001,BEEF CHUCK TENDER (NZ-PS),2023-05-30
17,CCUR07,CHICKEN- CURRY CHICKEN 10kg/ctns,2023-06-15
19,CSLI02-03,CHICKEN SLICED W/O SKIN 2CM 2kgX6 kt,2023-02-06
20,CWHL15,CHICKEN- WHOLE LEG 500gm/pkt,2023-06-15
79,FSB002-02,FISH- FRESH SEABASS 700-800GM (MALAYSIA),2022-04-07
...,...,...,...
3201,PM-02,PORK- MINCED PORK A 6X2KG/CTNS,2023-06-13
3202,PRIB06,PORK- SPARERIB CUT 1.5''X1.5'' 5KGX2PKT/CTNS,2023-06-15
3229,SAUCE-61,SAUCE- RENDANG PASTE 1KG/PKT,2021-07-14
3232,SQC002,SEAFOOD- SQUID CUT WITHOUT SKIN (AA) 12X1KG/CTNS,2023-02-15


# Item Codes in Sales Data Missing In Inventory Data

### Possibilities:
1. Product was sold and depleted inventory to 0 so product was stock out

In [237]:
check_sales_item_codes_missing_in_inv = sales_df.merge(inv_df,on='item_code',how='left')
display(check_sales_item_codes_missing_in_inv[check_sales_item_codes_missing_in_inv['last_purchase_date'].isna()][['item_code','description_x']].drop_duplicates())

,item_code,description_x
8,PRIB01-13,PORK MEALTY RIBLETS (AURORA) 10kg/ctn
36,MECHICKEN05,CAJUN SPICE CHICKEN BONELESS LEG 1PCS/PKT
42,FTN002,FISH- FRESH TUNA (INDONESIA MAGURO)
63,CWHL03,CHICKEN- WHOLE LEG (SEARA/LAR) 15kg/ctn
95,PBEL05-09,PORK BELLY SHEET RIBBED AURORA 25KG/CTNS
...,...,...
123073,PMPD01,PORK MEAT POWDER
123938,ODS001,OTHER- DOU JIE
125737,PLIV05,HEART PROTECTING MEAT (LIVER)
127002,PSOF02-03,"PORK SOFTBONE CUT 2""x 1"" _ 02 6kg/pkt"


# Item Codes in Purchase Data Missing In Inventory Data

### Possibilities:
1. Product was bought and sold rapidly and inventory went to 0 leading to stock out thus these are high turnover items.

In [238]:
check_purchases_item_codes_missing_in_inv = pur_pdt_df.merge(inv_df,on='item_code',how='left')
display(check_purchases_item_codes_missing_in_inv[check_purchases_item_codes_missing_in_inv['last_purchase_date'].isna()][['item_code','description_x']].drop_duplicates())

,item_code,description_x
3,CSBL01-03,CHICKEN- KAMPONG CHICKEN WHOLE LEG
11,CWIN01,CHICKEN WING 80-100gm (SEARA/PERDIGAO) 15kg/ctn
12,CMIJ03-01,CHICKEN- MID JOINT 30gm (PERDIGAO) 1kgX16pkt
22,FBA001-02,FISH- WHOLE BATANG 3/4KG (NSW-LH)
24,OSA005-03,OTHER- PORK SAUSAGE 6INCH 20X1KG/CTNS
...,...,...
3273,PSPA12,PORK- SPARERIB 18KG
3294,PLIV05,HEART PROTECTING MEAT (LIVER)
3381,FYT003-01,FISH- FRESH TAIXING WR (1-1.6 KG)
3382,FSN001,FISH- FRESH TOMAN FILLET SLICED (U)


# Sales Dashboard

In [255]:
ft_sales_pdt_agg = inv_df.merge(pdt_sales_summary,on='item_code',how='outer')
ft_sales_pdt_agg['description'] = ft_sales_pdt_agg["description_x"].fillna(ft_sales_pdt_agg["description_y"])
ft_sales_pdt_agg.drop(columns=['description_x','description_y'],inplace=True)
ft_sales_pdt_agg.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_pdt_agg.csv"),index=False)

# daily_sales_price_upsample
# weekly_sales_price_upsample
# monthly_sales_price_upsample
# daily_sales_price
# monthly_sales_price

# weekly_sales_price.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_pdt_sales_price_ts.csv"),index=False)
# pdt_purchase_summary.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_pdt_cost_summary.csv"),index=False)
# pdt_sales_summary_per_customer.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_pdt_sales_summary_per_customer.csv"),index=False)


# daily_sales_total_per_customer_upsample
# weekly_sales_total_per_customer_upsample
# monthly_sales_total_per_customer_upsample
# daily_sales_total_per_customer
# weekly_sales_total_per_customer
# monthly_sales_total_per_customer

# weekly_sales_total_per_customer_upsample.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_pdt_sales_total_per_customer_ts.csv"),index=False)

sales_with_cogs.to_csv(os.path.join(OUTPUT_DIR, "sales_with_cogs.csv"),index=False)
profit_loss_invoices.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_profit_loss.csv"),index=False)

,date,customer_code,customer_name,item_code,description,quantity,currency,selling_price,document_no,total,...,latest_total_profit,median_total_profit,avg_total_profit,max_total_profit,min_total_profit,latest_pc1,median_pc1,avg_pc1,max_pc1,min_pc1
0,2022-10-01,UCH0024,HOKIEN MIAN,PSKI01,PORK SKIN 5kg/pkt,5.0,SGD,1.5,22205832,7.5,...,2.5,3.5,3.607692,5.0,2.5,0.333333,0.466667,0.481026,0.666667,0.333333
1,2022-10-02,UCH0024,HOKIEN MIAN,PSKI01,PORK SKIN 5kg/pkt,5.0,SGD,1.5,22205833,7.5,...,2.5,3.5,3.607692,5.0,2.5,0.333333,0.466667,0.481026,0.666667,0.333333
2,2022-10-03,UCH0024,HOKIEN MIAN,PSKI01,PORK SKIN 5kg/pkt,5.0,SGD,1.5,22205834,7.5,...,2.5,3.5,3.607692,5.0,2.5,0.333333,0.466667,0.481026,0.666667,0.333333
3,2022-10-01,UCB0060,BEDOK PRAWN NOODLE,PSKI01,PORK SKIN 5kg/pkt,5.0,SGD,1.0,22208915,5.0,...,0.0,1.0,1.107692,2.5,0.0,0.000000,0.200000,0.221538,0.500000,0.000000
4,2022-10-01,UCP0058,PRAWN NOODLE,PSKI01,PORK SKIN 5kg/pkt,5.0,SGD,1.2,22208979,6.0,...,1.0,2.0,2.107692,3.5,1.0,0.166667,0.333333,0.351282,0.583333,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74482,2022-12-30,UCG0012,6IX 6IX RESTAURANT AND BAR PTE LTD,PLIV05,HEART PROTECTING MEAT (LIVER),5.0,SGD,6.5,22244715,32.5,...,9.0,9.0,9.000000,9.0,9.0,0.276923,0.276923,0.276923,0.276923,0.276923
74483,2022-12-31,UCO0074,ONLINE INFLUENCER,PCOL10-02,PORK COLLAR CS IBERICO STEAK CUT 20PK/CTN,1.0,SGD,8.8,22244378,8.8,...,0.2,0.2,0.200000,0.2,0.2,0.022727,0.022727,0.022727,0.022727,0.022727
74484,2022-12-30,UCA0099,ANG'S FAMILY FOOD ENTERPRISE PTE LTD,PSPA12,PORK- SPARERIB 18KG,450.0,SGD,4.4,22244747,1980.0,...,450.0,450.0,450.000000,450.0,450.0,0.227273,0.227273,0.227273,0.227273,0.227273
74485,2022-12-31,UCR0067,RAKU INU BISTRO & BAR PTE LTD,OSR002-04,OTHER- VEGETABLE MINI ROLL 20gX20pcX20pkt/ctns,10.0,SGD,3.5,22244847,35.0,...,14.0,14.0,14.000000,14.0,14.0,0.400000,0.400000,0.400000,0.400000,0.400000


In [253]:
ft_sales_pdt_with_profit = ft_sales_pdt_agg[~ft_sales_pdt_agg['total_profit'].isna()]
ft_sales_pdt_with_profit.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_pdt_with_profit.csv"),index=False)

In [266]:

pdt_purchase_price_summary = pdt_purchase_summary[['item_code','median_price','min_price','max_price']]
ft_purchase_price_per_supplier_ts = weekly_price_per_pdt_supplier.merge(pdt_purchase_price_summary, on='item_code', how='left').rename(columns={"median_price": "group_average_price", "max_price": "group_max_price", "min_price": "group_min_price"})
ft_purchase_price_per_supplier_ts.to_csv(os.path.join(OUTPUT_DIR, "ft_purchase_price_per_supplier_ts.csv"),index=False)

,vendor_code,vendor_name,item_code,description,purchase_count,avg_price,min_price,median_price,max_price,total,total_qty,latest_purchase_date,group_median_price,group_max_price,group_latest_price,num_suppliers
0,USVS0018,SINGYI F&B PTE LTD,MECHICKEN03,MARINATE CHICKEN MID JOINT 5KG/PKT,10.0,7.800000,7.80,7.80,7.80,390.0,50.0,2022-12-26,7.80,7.80,7.80,1
1,USVS0018,SINGYI F&B PTE LTD,MEPORK01,MARINATE PORK BELLY,4.0,5.700000,5.70,5.70,5.70,159.6,28.0,2022-12-06,5.70,5.70,5.70,1
2,USVS0018,SINGYI F&B PTE LTD,OPN001,OTHER- PRAWN NUTS 1kg/pkt,31.0,12.509677,12.30,12.30,12.80,1484.4,118.0,2022-12-31,12.30,12.80,12.80,1
3,USVS0018,SINGYI F&B PTE LTD,OYR001,OTHER- FRIED YAM RING 230gm,5.0,3.800000,3.80,3.80,3.80,95.0,25.0,2022-12-29,3.80,4.00,4.00,3
4,UVA0001,ANGLISS SINGAPORE PTE LTD,BFC005,BEEF FATS 20KG/CTNS (HARVEY-AUST),1.0,3.600000,3.60,3.60,3.60,360.0,100.0,2022-10-29,3.55,3.60,3.50,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,UVY0006,YOCORN FOOD ENTERPRISE PTE LTD,OWS0002,OTHER- WASABI MINI PKT 2.6GM/SACHETS,1.0,0.050000,0.05,0.05,0.05,50.0,1000.0,2022-11-23,0.05,0.05,0.05,1
762,UVY0006,YOCORN FOOD ENTERPRISE PTE LTD,SCA001-07,SEAFOOD- SCALLOP HOTATE KAIBASHIDA SIZE 3S,1.0,39.000000,39.00,39.00,39.00,390.0,10.0,2022-10-12,39.00,39.00,39.00,1
763,UVZ0003,ZUZEN FOOD PTE LTD,OCL014,OTHER- MOUSSE CAKE - RED VELVET,1.0,20.000000,20.00,20.00,20.00,20.0,1.0,2022-12-24,20.00,20.00,20.00,1
764,UVZ0003,ZUZEN FOOD PTE LTD,SCP001,SEAFOOD- CUTTLEFISH PASTE 1kg x 30pkt,9.0,5.900000,5.70,5.90,6.50,5546.0,940.0,2022-12-24,5.90,6.50,5.90,1


## Procurement Dashboard

In [155]:
# weekly_sales_qty_upsample.to_csv(os.path.join(OUTPUT_DIR, "ft_sales_qty_ts.csv"),index=False)

supplier_items_summary.to_csv(os.path.join(OUTPUT_DIR, "ft_procurement_supplier_pdt_breakdown.csv"),index=False)

# ft_procurement_weekly_price_ts = weekly_sales_price_upsample.merge(weekly_price_upsample, on=['date','item_code','description'], how="left")
# ft_procurement_weekly_price_ts.rename(columns={"selling_price":"sales_price", "price":"purchase_price"}, inplace=True)
# ft_procurement_weekly_price_ts.to_csv(os.path.join(OUTPUT_DIR, "ft_procurement_weekly_price_ts.csv"),index=False)

# weekly_pv_per_pdt_per_supplier.to_csv(os.path.join(OUTPUT_DIR, "ft_procurement_weekly_pv_by_supplier_ts.csv"),index=False)

# Supplier Analaysis Dashboard

In [186]:
supplier_summary.to_csv(os.path.join(OUTPUT_DIR, "ft_supplier_summary.csv"),index=False)
# weekly_pv_per_supplier_upsample.to_csv(os.path.join(OUTPUT_DIR, "ft_supplier_weekly_pv_ts.csv"),index=False)

In [ ]:
item_code = weekly_sales_total_per_customer['item_code'].sample(1).values[0]
print(item_code)
test_df = weekly_sales_total_per_customer[weekly_sales_total_per_customer['item_code'] == item_code]

In [106]:
pdt_sales_summary[pdt_sales_summary['item_code'] == 'CSKI01']

,item_code,description,avg_sales_price,min_sales_price,median_sales_price,max_sales_price,latest_sales_price,latest_sales_date,total_revenue,total_sales_qty,total_profit,avg_pc1,min_pc1,median_pc1,max_pc1,pc1_loss,pc1_loss_invoice_cnt,num_invoices,num_unique_customers,avg_monthly_sales_qty
143,CSKI01,CHICKEN SKIN 15kg/ctn,2.5,2.5,2.5,2.5,2.5,2022-11-10,65.0,26.0,52.0,0.8,0.8,0.8,0.8,NaN,NaN,6,2,13.0
